# Lab2 - PuLP Library

<b>Information on group members:</b><br>
1) 156039 Krzysztof Skrobała <br>
2) 156034 Wojciech Bogacz

In [2]:
!pip install pulp
!pip install numpy
!pip install pandas



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [3]:
from pulp import *  
import numpy as np
import pandas as pd

## Introduction - brief tutorial on PuLP

In [4]:
# Create an LpProblem instance; LpMaximize = objective function is to be maximized
model = LpProblem(name="some-problem", sense=LpMaximize)

In [5]:
# Initialize the decision variables. We can set the name and lower bound as well.
# To create an array of variables, you can use comprehensions of LpProblem.dicts.

x1 = LpVariable(name="x1", lowBound=0)
x2 = LpVariable(name="x2", lowBound=0)

In [6]:
# An example expression
expression = 3 * x1 + 2 * x2
type(expression)

pulp.pulp.LpAffineExpression

In [7]:
# An example constraint
constraint = 2 * x1 + 3 * x2 >= 5
type(constraint)

pulp.pulp.LpConstraint

Ok, let's now use PuLP to solve the below problem:
$max$ $4x_1 + 2x_2$ <br>
s.t.<br>
     $1x_1 + 1x_2 \geq 1$ <br>
     $2x_1 + 1x_2 \leq 6$ <br>
     $-1x_1 + x_2 = 1$ <br>
     $x_1 \geq 0$ <br>
     $x_2 \geq 0$ 

In [8]:
# Problem
model = LpProblem(name="test-problem", sense=LpMaximize)

x1 = LpVariable(name="x1", lowBound=0)
x2 = LpVariable(name="x2", lowBound=0)

model += (1 * x1 + 1*x2 >= 1, "#1 constraint")
model += (2 * x1 + 1*x2 <= 6, "#2 constraint")
model += (-1 * x1 + 1*x2 == 1, "#3 constraint")

# Objective function
obj_func = 4*x1 + 2 * x2
model += obj_func

model

test-problem:
MAXIMIZE
4*x1 + 2*x2 + 0
SUBJECT TO
#1_constraint: x1 + x2 >= 1

#2_constraint: 2 x1 + x2 <= 6

#3_constraint: - x1 + x2 = 1

VARIABLES
x1 Continuous
x2 Continuous

In [9]:
# Solve the problem
status = model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/krzys/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/75c07f81d16f4cb9b85bd566acddd516-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/75c07f81d16f4cb9b85bd566acddd516-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 17 RHS
At line 21 BOUNDS
At line 22 ENDATA
Problem MODEL has 3 rows, 2 columns and 6 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-3) rows, 0 (-2) columns and 0 (-6) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 12
After Postsolve, objective 12, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 12 - 0 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



In [10]:
# Print status
print(f"status: {model.status}, {LpStatus[model.status]}")

status: 1, Optimal


In [11]:
# Print objective value
print(f"objective: {model.objective.value()}")

objective: 12.000000199999999


In [12]:
# Print constraint values
for name, constraint in model.constraints.items():  print(f"{name}: {constraint.value()}")

#1_constraint: 3.3333334
#2_constraint: 9.999999983634211e-08
#3_constraint: 0.0


In [13]:
model.variables()

[x1, x2]

In [14]:
print(x1.value())
print(x2.value())

1.6666667
2.6666667


### The same code but using dictionaries and other nice tricks


In [15]:
model = LpProblem(name="some-problem", sense=LpMaximize)

In [16]:
var_names = ["First", "Second"]
x = LpVariable.dicts("x", var_names, 0)
x

{'First': x_First, 'Second': x_Second}

In [17]:
const_names = ["GE", "LE", "EQ"]
sense = [1, -1, 0] # GE, LE, EQ
coefs = [[1,1],[2,1],[-1,1]] # Matrix coefs
rhs = [1, 6, 1] 

for c, s, r, cn in zip(coefs, sense, rhs, const_names):
    expr = lpSum([x[var_names[i]] * c[i] for i in range(2)])
    model += LpConstraint(e=expr, sense = s, name = cn, rhs = r)
    
obj_coefs = [4,2]
model += lpSum([x[var_names[i]] * obj_coefs[i] for i in range(2)])
    
model

some-problem:
MAXIMIZE
4*x_First + 2*x_Second + 0
SUBJECT TO
GE: x_First + x_Second >= 1

LE: 2 x_First + x_Second <= 6

EQ: - x_First + x_Second = 1

VARIABLES
x_First Continuous
x_Second Continuous

In [18]:
status = model.solve()
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
for n in var_names: print(x[n].value())

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/krzys/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/2b0f70a67d3140928a604ea48f10aece-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/2b0f70a67d3140928a604ea48f10aece-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 17 RHS
At line 21 BOUNDS
At line 22 ENDATA
Problem MODEL has 3 rows, 2 columns and 6 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-3) rows, 0 (-2) columns and 0 (-6) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 12
After Postsolve, objective 12, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 12 - 0 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

status

# Homework - use the PuLP library to solve the following OR problem 

Johnson & Sons company manufactures 6 types of toys. Each toy is produced by utilizing at least one Machine 1-4, requiring a different production time (see Table below). For instance, product A requires 4 minutes on Machine 1, 4 minutes on Machine 2, 0 Minutes on Machine 3, and 0 minutes on Machine 4 (all machines must be utilized to produce a toy unless the production time equals 0). Each machine is available for a different number of hours per week. The company aims to identify the number (product-mix) of toys that must be manufactured to maximize the income (can be continuous). Notably, each toy can be sold for a different price. Due to market expectations, the company wants to manufacture twice as many F toys as A. Furthermore, the number of toys B should equal C. Solve this problem using the PuLP library. Prepare a report (in the jupyter notebook) containing information on the following:
<ol>
<li>The number of toys to manufacture;</li>
<li>The expected income;</li>
<li>The production time required on each machine;</li>
</ol>
Consider the total and partial values, i.e., considered for each toy A-F separately (e.g., income resulting from selling toy A). Also, answer the following questions concerning the found solution:
<ol>
<li>Which toy(s) production should be focused on?  </li>
<li>Is there a toy that can be excluded from consideration for production? </li>
<li>Is there a machine that is not fully utilized?</li>
</ol>

| Toy | Machine 1 | Machine 2 | Machine 3 | Machine 4 | Selling price |
| --- | --- | --- | --- | --- | --- |
| A | 4 (minutes!) | 4 | 0 | 0 | 2.50 USD |
| B | 0 | 3 | 3 | 0 | 1.00 USD |
| C | 5 | 0 | 2 | 5 | 4.00 USD |
| D | 2 | 4 | 0 | 4 | 3.00 USD |
| E | 0 | 4 | 5 | 2 | 3.50 USD |
| F | 2 | 2 | 1 | 1 | 4.00 USD |
| Production time limit (hours!) | 120 | 60 |  80 |  120 |  |

# Defining the Problem
We define our problem in terms of __Linear Programming__ as follows:

$$ \textbf x = (x_a, x_b, x_c, x_d, x_e, x_f) $$
$$ \textbf p = (2.5,1,4,3,3.5,4)$$
$$ \max_\textbf x z = \textbf x \cdot \textbf p $$

Subject to constraints:

$$x_f = 2x_a$$
$$x_b = x_c$$
$$\textbf x \cdot \textbf m_1 \le 120 * 60$$
$$\textbf x \cdot \textbf m_2 \le 60 * 60$$
$$\textbf x \cdot \textbf m_3 \le 80 * 60$$
$$\textbf x \cdot \textbf m_4 \le 120 * 60$$


Where:
$$ \textbf m_1 = (4,0,5,2,0,2) $$
$$ \textbf m_2 = (4,3,0,4,4,2) $$
$$ \textbf m_3 = (0,3,2,0,5,1) $$
$$ \textbf m_4 = (0,0,5,4,2,1) $$

In [19]:
x = [LpVariable(name=f"x{i}", lowBound=0,cat='Integer') for i in ['A', 'B', 'C', 'D', 'E', 'F']]
model = LpProblem(name="Johnson&Son-problem", sense=LpMaximize)
price = [2.5, 1.0, 4.0, 3.0, 3.5, 4.0]
m1 = [4,0,5,2,0,2]
m2 = [4,3,0,4,4,2]
m3 = [0,3,2,0,5,1]
m4 = [0,0,5,4,2,1]


In [20]:
total_income = lpSum([price[i] * x[i] for i in range(6)])

In [21]:
model+=total_income

In [22]:
model+= (x[5]==2*x[0], "F==2A")
model+= (x[1]==x[2], "B==C")
model+= (x[0]*m1[0]+x[1]*m1[1]+x[2]*m1[2]+x[3]*m1[3]+x[4]*m1[4]+x[5]*m1[5]<=120*60, "m1")
model+= (x[0]*m2[0]+x[1]*m2[1]+x[2]*m2[2]+x[3]*m2[3]+x[4]*m2[4]+x[5]*m2[5]<=60*60, "m2")
model+= (x[0]*m3[0]+x[1]*m3[1]+x[2]*m3[2]+x[3]*m3[3]+x[4]*m3[4]+x[5]*m3[5]<=80*60, "m3")
model+= (x[0]*m4[0]+x[1]*m4[1]+x[2]*m4[2]+x[3]*m4[3]+x[4]*m4[4]+x[5]*m4[5]<=120*60, "m4")

In [23]:
model

Johnson&Son-problem:
MAXIMIZE
2.5*xA + 1.0*xB + 4.0*xC + 3.0*xD + 3.5*xE + 4.0*xF + 0.0
SUBJECT TO
F==2A: - 2 xA + xF = 0

B==C: xB - xC = 0

m1: 4 xA + 5 xC + 2 xD + 2 xF <= 7200

m2: 4 xA + 3 xB + 4 xD + 4 xE + 2 xF <= 3600

m3: 3 xB + 2 xC + 5 xE + xF <= 4800

m4: 5 xC + 4 xD + 2 xE + xF <= 7200

VARIABLES
0 <= xA Integer
0 <= xB Integer
0 <= xC Integer
0 <= xD Integer
0 <= xE Integer
0 <= xF Integer

In [24]:
status = model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/krzys/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/8fc363861e374d308e1857b015749375-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/8fc363861e374d308e1857b015749375-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 11 COLUMNS
At line 51 RHS
At line 58 BOUNDS
At line 65 ENDATA
Problem MODEL has 6 rows, 6 columns and 21 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 5700 - 0.00 seconds
Cgl0004I processed model has 4 rows, 4 columns (4 integer (0 of which binary)) and 14 elements
Cutoff increment increased from 1e-05 to 0.4999
Cbc0012I Integer solution of -5687.5 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
Cbc0038I Full problem 4 rows 4 columns, reduced to 3 rows 2 columns
Cbc0012I Integer solution of -56

In [25]:
LpStatus[model.status]

'Optimal'

In [26]:
x_values = np.array([v.value() for v in  x])

# Number of toys to manufacture

In [27]:
# print values of variables
for i in range(6):
    print(f"Optimal number of toys {str(x[i])[-1]} to manufacture: {x[i].value()}")

Optimal number of toys A to manufacture: 106.0
Optimal number of toys B to manufacture: 917.0
Optimal number of toys C to manufacture: 917.0
Optimal number of toys D to manufacture: 0.0
Optimal number of toys E to manufacture: 0.0
Optimal number of toys F to manufacture: 212.0


# Expected Total Income

In [28]:
print(f"Expected income is {model.objective.value()} USD")

Expected income is 5698.0 USD


# Expected income from each toy

In [29]:
for i in range(6):
    print(f"Expected income from toy {str(x[i])[-1]} is {x[i].value() * price[i]}")

Expected income from toy A is 265.0
Expected income from toy B is 917.0
Expected income from toy C is 3668.0
Expected income from toy D is 0.0
Expected income from toy E is 0.0
Expected income from toy F is 848.0


# The production time required on each machine

In [30]:
print(f"The machine number 1 will be manufacturing for: {x_values@np.array(m1)/60} hours ({x_values@np.array(m1)} minutes)")
print(f"The machine number 2 will be manufacturing for: {x_values@np.array(m2)/60} hours ({x_values@np.array(m2)} minutes)")
print(f"The machine number 3 will be manufacturing for: {x_values@np.array(m3)/60} hours ({x_values@np.array(m3)} minutes)")
print(f"The machine number 4 will be manufacturing for: {x_values@np.array(m4)/60} hours ({x_values@np.array(m4)} minutes)")

The machine number 1 will be manufacturing for: 90.55 hours (5433.0 minutes)
The machine number 2 will be manufacturing for: 59.983333333333334 hours (3599.0 minutes)
The machine number 3 will be manufacturing for: 79.95 hours (4797.0 minutes)
The machine number 4 will be manufacturing for: 79.95 hours (4797.0 minutes)


# Which toy(s) production should be focused on?

In [33]:
for i in range(6):
    print(f"Optimal number of toys {str(x[i])[-1]} to manufacture is {x[i].value()} with income {x[i].value() * price[i]}")

Optimal number of toys A to manufacture is 106.0 with income 265.0
Optimal number of toys B to manufacture is 917.0 with income 917.0
Optimal number of toys C to manufacture is 917.0 with income 3668.0
Optimal number of toys D to manufacture is 0.0 with income 0.0
Optimal number of toys E to manufacture is 0.0 with income 0.0
Optimal number of toys F to manufacture is 212.0 with income 848.0


#### Clearly the most profitable toy is Toy C, which makes it a toy to put the most focus on

# Is there a toy that can be excluded from consideration for production?

In [36]:
for i in range(6):
    if x[i].value() == 0:
        print(f"Toy {str(x[i])[-1]} can be excluded, as the optimal amount to produce is 0")

Toy D can be excluded, as the optimal amount to produce is 0
Toy E can be excluded, as the optimal amount to produce is 0


# Is there a machine that is not fully utilized?

In [44]:
print(f"The machine number 1 is utilized in {x_values@np.array(m1)/(60*120):.0%}")
print(f"The machine number 2 is utilized in {x_values@np.array(m2)/(60*60):.0%}")
print(f"The machine number 3 is utilized in  {x_values@np.array(m3)/(60*80):.0%}")
print(f"The machine number 4 is utilized in  {x_values@np.array(m4)/(60*120):.0%}")

The machine number 1 is utilized in 75%
The machine number 2 is utilized in 100%
The machine number 3 is utilized in  100%
The machine number 4 is utilized in  67%
